> #### 1. 그래프로 표현될 수 있는 데이터
> #### 2. 그래프 데이터에서의 예측 문제 종류, 그래프 표현법
> #### 3. GNN
> #### 4. GNN의 실제 데이터 적용
> #### 5. GNN 관련 다양한 주제들

## 4. 실제 데이터로 알아보는 GNN

그래프 단위 예측 문제 - Leffingwell의 냄새 데이터셋
- 여러 분자들에 대해 그 분자가 내는 냄새를 라벨링한 데이터셋
- 문제를 간단하게 하기 위해, 결과를 이진화 함. 분자가 톡 쏘는(pungent) 냄새를 내는지 아닌지로만 구별.
- 예시: 마늘, 머스타드의 allyl alcohol, 페퍼민트 맛 캔디의 piperitone

분자를 그래프로 표현
- 원자를 Node로, 공유결합을 Edge로
- 각 원자가 무엇인지(탄소, 질소, 산소 등등)와 각 공유결합의 종류가 무엇인지(단일, 이중, 삼중, 방향족 등)를 One-hot encoding으로 표현
- 예시: 물 분자  

<img src="imgs/water_mol.jpg" alt="water_molecule" width="400"/>  

원자

|   | H | O | C | N |
|---|---|---|---|---|
| 1 | 1 | 0 | 0 | 0 |
| 2 | 1 | 0 | 0 | 0 |
| 3 | 0 | 1 | 0 | 0 |

공유결합

|   | Single | Double | Triple |
|:-:|:------:|:------:|:------:|
| 1 |    1   |    0   |    0   |
| 2 |    1   |    0   |    0   |

### GNN 디자인

<img src="imgs/overall_gnn.png" alt="overall_gnn"/>


- GNN 모델을 디자인할 때 고려할 것
1. GNN 레이어 갯수, 깊이(depth)
2. 각 요소(attribute; node, edge, graph)의 각 레이어에서의 업데이트 이후 차원. 각 레이어의 업데이트 함수는 ReLu 활성화를 이용하는 MLP(Multi Layer Perceptron)  
<img src="imgs/mlp.png" alt="mlp"/>  

n= 입력 요소의 차원/ m=업데이트 된 요소의 차원

3. 통합 함수(aggregation funtion): max, mean, sum
4. 업데이트될 요소의 종류, 즉 메시지 패싱 유형: node, edge, graph 각각의 embedding을 메시지로 활용할 것인지.
> graph 메시지 패싱  
> :모든 node와 edge에 연결되어 있는 **master node**를 도입해서, 멀리 떨어진 node 또는 edge 간에도 정보가 공유될 수 있게함.
<img src="imgs/graph_message_passing.png" alt="graph_message_passing" width="400"/>  

### GNN 디자인에 따른 성능

GNN 레이어를 모두 거친 최종 graph embedding을 PCA를 통해 2차원으로 축소시켜 시각화  
graph embedding에 대한 linear classification 결과의 AUC(Area Under the Curve) 수치

#### Parameter 수 🌐
Trainable 변수의 개수가 증가할 수록, 성능이 좋아짐

#### Embedding 차원 🌐
Embedding의 차원이 높아질 수록, 성능이 좋아지는 경향. Parameter 수와 밀접한 관련이 있으므로, 합리적인 결과.  
그러나 각 차원의 가장 좋은 성능의 모델의 경우, 저차원임에도 고차원 모델보다 더 좋은 경우도 있음  

#### GNN 레이어 수 🌐
Layer의 수가 많아질 수록 성능이 좋아지는 경향. 마찬가지로 레이어 수가 적음에도 더 좋은 성능을 나타내는 경우도 있음  
또한 4겹 레이어의 가장 안좋은 성능의 모델이 2,3겹보다 떨어지는 현상. 레이어가 늘어나면서 지역적인 정보가 더 넓은 범위까지 퍼져나가게 되고, 기존에 갖고 있던 경향성이 레이어를 지날수록 희석되는 현상이 나타날 수 있음.

#### 통합 함수 🌐
sum을 사용할 때 다른 함수보다 아주 근소하게 좋았지만, 거의 비슷했음

#### 메시지 패싱 유형 🌐
전반적으로 더 많은 요소들간의 정보가 교환되는 모델일수록, 평균적으로 더 좋은 성능을 나타냄  
현재 주어진 과제가 그래프 단위 과제이므로, global 요소를 업데이트하는 것이 가장 좋은 성능 개선을 보임  
또한 분자 그래프에서 node가 의미하는 것(원자의 종류)이 edge(공유결합의 종류) 보다 냄새에 더 많은 영향을 끼치므로 node 요소를 업데이트하는 것도 큰 개선을 보임

#### 추가적인 성능 향상을 위한 방향
1. 정보 교환 방식
Linear graph path와 같은 구조에서는 이웃성을 기반으로 한 Pooing 방법이 잘 작동하지 않음. 관련 연구 활발
<img src="imgs/linear_graph.png" alt="linear_graph" width="400"/>  

2. 더 많은 feature를 그래프에 제공
embedding 차원이 늘어날수록 좋은 성능 &rarr; feature를 추가해 줌  
예시: 원자간 거리 등  
<img src="imgs/water_distance.png" alt="water_distance" width="400"/>  

## 5. GNN 관련 다양한 주제들

### 다른 종류의 그래프

message passing 프레임워크는 다른 종류의 그래프에도 그대로 적용이 가능.  
#### Multigraph  
예시: social network에서 edge 유형별로 다른 그래프를 그릴 수 있고, 이 그래프들을 통합하면 multigraph
<img src="imgs/social_net_multigraph.png" alt="social_net_multigraph" width="800"/>

각각의 edge 유형별 그래프에서 별도로 업데이트하면 됨

#### Hypernode graph
하나의 그래프를 하나의 노드(Hypernode)로 간주해서, 그래프의 그래프를 그릴 수 있음. 계층적 정보를 포함.  
예시: 분자간의 상호작용을 표현하는 그래프를 그릴 경우, 이 그래프의 node는 하위 단계의 그래프를 표현하는 hypernode

<img src="imgs/hypernode.png" alt="hypernode" width="400"/>

각 단계를 오가면서 업데이트하면 됨

### 배치 학습법
그래프의 크기가 너무 커서 메모리에 올릴 수 없거나 학습시간이 오래 걸릴때, 그래프의 일부만 사용하여 학습  
일반 NN과 다른 점은, 이웃 node와 edge의 개수가 달라서 일률적인 크기의 배치 사이즈를 정할 수가 없다는 것.  
&rarr; **핵심적인 정보를 담고 있는 subgraph**를 추출해 내어 학습에 이용  
핵심적인 정보라는 것은 문맥에 따라 다름.  

<img src="imgs/different_sampling.png" alt="different_sampling" width="600"/>

### 통합 연산(Aggregation operation)
바람직한 통합 함수는 유사한 입력에 대해 유사한 결과값을 낼 수 있어야 하고, 그 반대여야 하기도 함. 또한 입력 데이터에 순서가 있을 경우 어떤 순서로 입력되더라도 같은 결과값을 내야 함. 대표적으로 max, mean, sum 🌐  
상황에 따라 최적의 선택은 달라짐
- mean: 이웃의 갯수가 다르고, 정규화된 관점으로 feature를 보아야 할 때
- max: outlier를 강조해야 할 때
- sum: mean과 max의 균형. 흔히 쓰임

### Subgraph 생성자로서의 GCN
GCN에서 k개의 레이어를 거치면 특정 node의 입장에서 k 거리 이하에 존재하는 노드들로 구성된 subgraph가 있고, 최초 node의 최종 embedding은 각 subgraph의 graph embedding으로 간주할 수 있음. 이런 과정 없이 임의로 subgraph를 추출하면 모든 조합을 시도해야하므로 계산적으로 매우 비쌈. GCN을 이용한 방법으로 모든 subgraph에 대한 연산 결과에 대한 근사값을 얻을 수 있음

<img src="imgs/gcn_subgraphs.png" alt="gcn_subgraphs" width="800"/>

### Node, Edge 그래프 duality
<img src="imgs/dual.png" alt="dual" width="800"/>


### 행렬곱을 이용한 Graph convolution
인접행렬 $A$와 Node feature 행렬 $X$의 곱 $B=XA$는 이웃 Node의 정보를 통합한 결과값  
$B_{ij} = A_{i,1}X_{1,j}+A_{i,2}X_{2, j}+ \cdot\cdot\cdot +A_{i,n}X_{n, j}=\sum_{A_{i,k}>0} X_{k,j}$  
i번째 node의 j번째 feature의 통합 결과값

sparse한 인접행렬의 경우, 불필요한 (비싼) 행렬연산을 많이 하게됨 &rarr; 인접리스트 사용!

#### 또다른 행렬곱 활용법: walks on graph
인접행렬 $A$를 $K$번 곱하면 $A^K_{ij}$는 i번째 node에서 j번째 node까지 가는 경로의 수를 모두 카운트한 값  
$A^2_{ij} = A_{i,1}A_{1, j}+A_{i,2}A_{2, j}+ \cdot\cdot\cdot +A_{i,n}A_{n,j}$  
$A^3_{ij} = A^2_{i,1}A_{1, j}+A^2_{i,2}A_{2, j}+ \cdot\cdot\cdot +A^2_{i,n}A_{n,j}$  
$\cdot\cdot\cdot$  
$A^K_{ij} = A^{K-1}_{i,1}A_{1, j}+A^{K-1}_{i,2}A_{2, j}+ \cdot\cdot\cdot +A^{K-1}_{i,n}A_{n,j}$  

### Graph Attention Network
이웃노드를 모두 같게 대하지 않고, 유사한 노드에 더 가중치를 주는 것  
노드간의 유사도를 weight로 줌  
<img src="imgs/attention.png" alt="attention" width="600"/>

### 설명가능한 GNN
모델이 예측을 할 때 어떤 feature 혹은 어떤 parameter이 중요한 역할을 하는가? (**설명가능성**explainability)&rarr; 모델의 신뢰도, 디버깅의 용이함, 과학적 발견의 가능성을 높임  
예시: 톡 쏘는 냄새를 내는 분자(graph)와 그렇지 않은 분자 사이에 특정 원자 그룹(subgraph)의 유무 차이가 있는가?  
GNNExplainer - 과제와 가장 관련있는 subgraph를 알아냄  
Attribution technique - 그래프의 각 node와 edge에 대해 과제와의 관련도에 따라 랭크를 매김
<img src="imgs/attribution.png" alt="attribution" width="600"/>

### Generative 모델링
주어진 그래프들로부터 패턴을 학습한 뒤, 그로부터 새로운 그래프를 생성하는 것  
예시: 존재하지 않는 분자를 합성하여 신약으로 개발하는 경우  

GraphVAE - NN의 Autoencoder처럼 차원 축소를 통해 잠재된(latent) 패턴을 추출한 뒤, 이를 기반으로 node 간 edge 존재 여부를 예측하여 새로운 그래프를 만들어냄
<img src="imgs/graphvae.png" alt="graphvae" width="600"/>
